In [1]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Reshape
from keras.utils import to_categorical
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dropout, InputLayer, LSTM
from keras.layers import Bidirectional, BatchNormalization
from keras.layers import TimeDistributed
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
from keras.layers import Conv1D, AveragePooling1D, Dropout, UpSampling1D, BatchNormalization, LeakyReLU
from keras.regularizers import l1_l2
from sklearn.metrics import f1_score
from collections import Counter
from IPython import embed
import random
import statistics as st
from scipy import stats
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Bidirectional, LSTM
from tensorflow.python.keras import regularizers
from keras import backend as K

from sklearn.metrics import classification_report

In [2]:
train_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/'
test_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/'

In [3]:

all_x_files = sorted(glob.glob(train_path + "/subject_00*_*__x.csv"))
all_x_time_files = sorted(glob.glob(train_path + "/subject_00*_*__x_time.csv"))
all_y_files = sorted(glob.glob(train_path + "/subject_00*_*__y.csv"))
all_y_time_files = sorted(glob.glob(train_path + "/subject_00*_*__y_time.csv"))

In [4]:
num_test = 2
num_val = 2

subject_list_x_test = all_x_files[:num_test]
subject_list_x_val = all_x_files[num_test:num_test+num_val]
subject_list_x_train = all_x_files[num_test+num_val:]

# print the train, val, and test lists
print("Training subjects:", subject_list_x_train)
print("Validation subjects:", subject_list_x_val)
print("Testing subjects:", subject_list_x_test)


Training subjects: ['/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_05__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_06__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_07__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_08__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_01__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_02__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_03__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_04__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_05__x.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecogniti

In [5]:
num_test = 2
num_val = 2

subject_list_y_test = all_y_files[:num_test]
subject_list_y_val = all_y_files[num_test:num_test+num_val]
subject_list_y_train = all_y_files[num_test+num_val:]

# print the train, val, and test lists
print("Training subjects:", subject_list_y_train)
print("Validation subjects:", subject_list_y_val)
print("Testing subjects:", subject_list_y_test)

Training subjects: ['/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_05__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_06__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_07__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_08__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_01__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_02__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_03__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_04__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_002_05__y.csv', '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecogniti

In [6]:
subject_list_y_test_time = all_y_time_files[:num_test]
subject_list_y_test_time

['/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_01__y_time.csv',
 '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/subject_001_02__y_time.csv']

In [7]:
def upsampling(X_file, y_file):
    df_X = pd.read_csv(X_file)
    df_y = pd.read_csv(y_file)
    print("df_X:", df_X.shape)
    print("df_Y:", df_y.shape)
    new_labels = []
    for label in df_y.iterrows():
        new_labels += [label[1][0]] * 4
    
    new_labels_df = pd.DataFrame(new_labels)
    end_row = df_X.shape[0] - new_labels_df.shape[0]
    df_X = df_X.iloc[:-end_row,:]
    return df_X, new_labels_df

In [8]:
def scale_data(input_data, columns):
    scaler = StandardScaler()
    scaler = scaler.fit(input_data[columns])
    input_data.loc[:, columns] = scaler.transform(input_data[columns].to_numpy())
    return input_data 

In [9]:
def take_max_labels(X, y, window_size, step_size):
    X_values = []
    y_values = []
    for i in range(0, len(X) - window_size, step_size):
        value = X.iloc[i:(i + window_size)].values
        X_values.append(value)
        labels = y.iloc[i:(i + window_size)]
        y_values.append(stats.mode(labels)[0][0])
    return np.array(X_values) ,np.array(y_values).reshape(-1, 1)

In [10]:
def one_hot_encoding(labels):
    training_y_encoded = to_categorical(labels, num_classes=4).reshape(-1,4)
    return training_y_encoded

In [11]:
def create_data(X_files, y_files, window_size, step_size):
    combined_X = []
    combined_y = []
    for i in range(len(X_files)):
        X, y = upsampling(X_files[i], y_files[i])
        X = scale_data(X, list(X.columns.values))
        X, y = take_max_labels(X, y, window_size, step_size)
        print("X_mode:",X.shape)
        print("Y mode", y.shape)
        y = one_hot_encoding(y)
        print("Y one hot", y.shape)
        combined_X.append(X)
        combined_y.append(y)
    return np.concatenate(combined_X), np.concatenate(combined_y)

In [13]:
x_train, y_train = create_data(subject_list_x_train, subject_list_y_train, 40, 1)

df_X: (59120, 6)
df_Y: (14779, 1)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (59076, 40, 6)
Y mode (59076, 1)
Y one hot (59076, 4)
df_X: (38630, 6)
df_Y: (9657, 1)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (38588, 40, 6)
Y mode (38588, 1)
Y one hot (38588, 4)
df_X: (31422, 6)
df_Y: (7855, 1)


...

X_mode: (44596, 40, 6)
Y mode (44596, 1)
Y one hot (44596, 4)
df_X: (39438, 6)
df_Y: (9859, 1)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (39396, 40, 6)
Y mode (39396, 1)
Y one hot (39396, 4)
df_X: (48137, 6)
df_Y: (12034, 1)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (48096, 40, 6)
Y mode (48096, 1)
Y one hot (48096, 4)


In [ ]:
x_val, y_val = create_data(subject_list_x_val, subject_list_y_val, 40, 1)

df_X: (43145, 6)
df_Y: (10786, 1)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (43104, 40, 6)
Y mode (43104, 1)
Y one hot (43104, 4)
df_X: (54955, 6)
df_Y: (13738, 1)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-10-44586919988f>:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_values.append(stats.mode(labels)[0][0])


X_mode: (54912, 40, 6)
Y mode (54912, 1)
Y one hot (54912, 4)


In [12]:
combined_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/combined_data/'

In [ ]:
np.save(combined_path + 'x_train.npy', x_train)
np.save(combined_path + 'y_train.npy', y_train)
np.save(combined_path + 'x_val.npy', x_val)
np.save(combined_path +'y_val.npy', y_val)

In [13]:
x_train = np.load(combined_path + 'x_train.npy')
y_train = np.load(combined_path + 'y_train.npy')
x_val = np.load(combined_path + 'x_val.npy')
y_val = np.load(combined_path +'y_val.npy')

In [14]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1134384, 40, 6) (1134384, 4)
(98016, 40, 6) (98016, 4)


In [15]:
label_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
label_weights = {i:label_weights[i] for i in range(len(label_weights))}
label_weights

{0: 0.3306717535178094,
 1: 6.970872354546125,
 2: 4.961354769860569,
 3: 1.5851853508026652}

In [16]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
def BiLSTM_model(training_X, training_y_encoded):
    n_timesteps, n_features, n_outputs = training_X.shape[1], training_X.shape[2], training_y_encoded.shape[1]
    model = Sequential()
    model.add(Bidirectional(LSTM(units = 64, return_sequences=True), input_shape = (n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Dropout(rate = 0.5))
    model.add(Bidirectional(LSTM(units = 64, return_sequences=True), input_shape = (n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Dropout(rate = 0.5))
    model.add(Bidirectional(LSTM(units = 32, return_sequences=True), input_shape = (n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Dropout(rate = 0.4))
    model.add(Flatten())
    model.add(Dense(units = 100, activation = 'relu'))
    model.add(Dense(units = 50, activation = 'relu'))
    model.add(Dense(n_outputs, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', f1_m])
    return model

In [18]:
model_lstm = BiLSTM_model(x_train, y_train)
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40, 128)          36352     
 l)                                                              
                                                                 
 batch_normalization (BatchN  (None, 40, 128)          512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 40, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 40, 128)          98816     
 nal)                                                            
                                                                 
 batch_normalization_1 (Batc  (None, 40, 128)          512       
 hNormalization)                                        

In [ ]:
with tf.device('/GPU:0'):
  lstm_fit = model_lstm.fit(x_train, y_train, epochs=20, verbose=1, batch_size = 128, class_weight = label_weights, validation_data=(x_val, y_val), shuffle = True)

In [21]:
def save_model(model):
    model_json = model.to_json()
    with open(combined_path + 'model_lstm.json', 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(combined_path + 'model_lstm.h5')
    print('Saved model to disk.')

In [22]:
save_model(model_lstm)

Saved model to disk.


In [23]:
def load_model(model_file_name, weight_file_name):
    json_file = open(model_file_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # Load weights into new model
    loaded_model.load_weights(weight_file_name)
    print('Loaded model from disk.')
    return loaded_model

In [24]:
model_lstm = load_model(combined_path + 'model_lstm.json', combined_path + 'model_lstm.h5')

Loaded model from disk.


In [25]:
def create_test_dataset(X, window_size, step_size):
    X_values = []
    for i in range(0, len(X) - window_size, step_size):
        value = X.iloc[i:(i + window_size)].values
        X_values.append(value)        
    return np.array(X_values)

In [26]:
def get_test_labels(output):
    new_labels = []
    for i in range(0, output.shape[0], 4):
        lst = list(output[i:i + 4])
        new_labels.append(stats.mode(lst)[0][0])  
    return np.array(new_labels).reshape(-1, 1)

In [27]:
def get_predictions(subject_list_x_test, subject_list_y_test,window_size, output_filenames):
  for i in range(len(subject_list_x_test)):
      input_data = pd.read_csv(subject_list_x_test[i])

      df = scale_data(input_data, list(input_data.columns.values))
      
      y_frame = pd.read_csv(subject_list_y_test[i])

      addl = y_frame.shape[0] * 4 - df.shape[0] + window_size

      addl_df = pd.DataFrame(df.iloc[-addl:])

      df = df.append(addl_df)
      X_test = create_test_dataset(df,window_size , 1)
      y_test = model_lstm.predict(X_test, batch_size = 128, verbose = 1)
      y_test_bool = np.argmax(y_test, axis = 1)
      y_actual = get_test_labels(y_test_bool)
      print(y_actual.size)
      y_df = pd.DataFrame(y_actual)
      y_df.to_csv(combined_path + output_filenames[i], index = False)

In [28]:
output_filenames_test = ['y_pred1.csv', 'y_pred2.csv']

In [29]:
combined_path

'/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/combined_data/'

In [30]:
window_size = 40

In [31]:
get_predictions(subject_list_x_test, subject_list_y_test,window_size, output_filenames_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-27-9fcf51075a30>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(addl_df)


296/296 [==============================] - 9s 9ms/step


<ipython-input-26-2244fece8eaf>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  new_labels.append(stats.mode(lst)[0][0])


9472


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-27-9fcf51075a30>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(addl_df)


549/549 [==============================] - 6s 7ms/step


<ipython-input-26-2244fece8eaf>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  new_labels.append(stats.mode(lst)[0][0])


17542


In [32]:
df = pd.read_csv(combined_path + 'y_pred1.csv')

In [33]:
df.value_counts()

0
0    5631
3    2776
1    1064
2       1
dtype: int64

In [34]:
test_1 = pd.read_csv(subject_list_y_test[0])
test_1.shape

(9472, 1)

In [35]:
y_pred = df.values.ravel()  
y_true = test_1.values.ravel() 
f1_macro = f1_score(y_true, y_pred, average='macro')

In [ ]:
f1_macro

In [37]:
df2 = pd.read_csv(combined_path + 'y_pred2.csv')

In [ ]:
test_2 = pd.read_csv(subject_list_y_test[1])
test_2.shape

In [ ]:
df2.shape

In [ ]:
y_pred = df2.values.ravel()  
y_true = test_2.values.ravel() 
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_macro

In [41]:
all_x_test_files = sorted(glob.glob(test_path + "/subject_0*_*__x.csv"))
all_x_test_time_files = sorted(glob.glob(test_path + "/subject_0*_*__x_time.csv"))
all_y_test_time_files = sorted(glob.glob(test_path + "/subject_0*_*__y_time.csv"))

In [42]:
all_y_test_time_files

['/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/subject_009_01__y_time.csv',
 '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/subject_010_01__y_time.csv',
 '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/subject_011_01__y_time.csv',
 '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/subject_012_01__y_time.csv']

In [43]:
output_filenames = ['subject_009_01__y.csv','subject_010_01__y.csv','subject_011_01__y.csv','subject_012_01__y.csv']

In [44]:
get_predictions(all_x_test_files, all_y_test_time_files, window_size, output_filenames)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-27-9fcf51075a30>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(addl_df)


297/297 [==============================] - 2s 8ms/step


<ipython-input-26-2244fece8eaf>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  new_labels.append(stats.mode(lst)[0][0])


9497


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-27-9fcf51075a30>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(addl_df)


...

12939


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-27-9fcf51075a30>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(addl_df)


355/355 [==============================] - 2s 7ms/step


<ipython-input-26-2244fece8eaf>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  new_labels.append(stats.mode(lst)[0][0])


11329


In [45]:
test = pd.read_csv(all_y_test_time_files[3], header=None)
test

,0
0,0.02
1,0.12
2,0.22
3,0.32
4,0.42
...,...
11325,1132.52
11326,1132.62
11327,1132.72
11328,1132.82


In [46]:
df2 = pd.read_csv(combined_path + 'subject_009_01__y.csv', header=None)

In [47]:
df2

,0
0,0
1,0
2,0
3,0
4,0
...,...
9493,0
9494,0
9495,0
9496,0


In [48]:
df2.value_counts()

0    7028
3    1681
1     787
2       2
dtype: int64

In [ ]:
plt.plot(df2)